# UR3


PS я в итоге немного перевыполнил задание так как в итоге не зная контент 11 домашки подбирал ещё и мотор, хоть и не по параметрам 11 лекции... прошу к этому отнестить как к доп ресерчу)))


подробнее про запуск самого решения в README



### 2.1. UR3 kinematic и inertial информация ищ URDF

Исходная модель UR3 берётся из URDF:

- предельные моменты (`effort`):
  - shoulder_pan / shoulder_lift: **330 N·m**
  - elbow: **150 N·m**
  - wrist_1 / wrist_2 / wrist_3: **54 N·m**
- предельные скорости (`velocity`):
  - плечи: **2.16 rad/s**
  - локоть: **3.15 rad/s**
  - запястья: **3.20 rad/s**

URDF также содержит массу и диагональные моменты инерции для звеньев:
`shoulder_link`, `upper_arm_link`, `forearm_link`,
`wrist_1_link`, `wrist_2_link`, `wrist_3_link`.

URDF → MuJoCo: модель экспортирована в MJCF (`mjmodel.xml`).

### 2.2. SHA-SG actuator catalog
я нашёл что для ur3 передаточная частота нетипичная 101:1 (https://people.ohio.edu/williams/html/PDF/UniversalRobotKinematics.pdf)
я прошерстил каталог, но большенство SHA имеют модели под 100
https://www.harmonicdrive.net/downloads/catalogs
Из каталога Harmonic Drive, серия **SHA‑SG** чуть ли не единственная модель, которая имела передаточное число **101:1**
для каждой модели берём:

- Maximum Torque, N·m
- Maximum Rotational Speed, rpm
- Moment of Inertia GD²/4, kg·m² (инерция относительно выходного вала)
- Mass (without brake), kg
- габариты:
  - `øA` → радиус цилиндра `R = øA / 2`
  - `E` → длина цилиндра `h`

Сводная таблица (используемые модели):

| Model      | Max torque (N·m) | Max speed (rpm) | Inertia GD²/4 (kg·m²) | Mass (kg) | Radius R (m) | Length h (m) |
|-----------|------------------|-----------------|------------------------|-----------|--------------|--------------|
| SHA20A-101 | 107 | 59.4 | 0.91 | 2.0 | 0.047 | 0.103 |
| SHA25A-101 | 204 | 55.4 | 2.2  | 5.0 | 0.057 | 0.109 |
| SHA32A-101 | 433 | 47.5 | 8.0  | 2.95| 0.073 | 0.125 |


## 3. Actuator sizing

### 3.1. Joint

Конвертируем предельные скорости UR3 в об/мин:

$[
n_{\mathrm{rpm}} = \dot q_{\max} \cdot \frac{60}{2\pi}.
$]

Вводим коэффициенты запаса:

- по моменту: $(k_\tau = 1.3$) (30 %)
- по скорости: $(k_\omega = 1.2$) (20 %)

Требуемые значения:

$[
\tau_{\text{need}} = k_\tau\,\tau_{\text{URDF}}, \quad
n_{\text{need}} = k_\omega\,n_{\text{URDF}}.
$]

Для трёх типов суставов (плечо / локоть / запястье)
мы получим числа, которые ниже в разделе 9.2 проверяются кодом.

### 3.2. Mapping SHA-SG models to UR3 joints

Сравниваем $(\tau_{\text{need}}$) и $(n_{\text{need}}$)
с значениями SHA‑SG:

- плечи требуют ~429 N·m → берём **SHA32A‑101** (433 N·m)
- локоть требует ~195 N·m → берём **SHA25A‑101** (204 N·m)
- запястья требуют ~70 N·m → берём **SHA20A‑101** (107 N·m)

это основываясь на  URDF с запасом который я раннее ввёл, однако в URDF судя по всему значения торка НЕ ПРАВИЛЬНЫЕ, так как основываясь на их официальной статье https://www.universal-robots.com/articles/ur/robot-care-maintenance/max-joint-torques-cb3-and-e-series/?utm_source=chatgpt.com эзначения в URDF для UR3 это значения для UR10 по статье
По скорости все актуаторы с запасом удовлетворяют требованиям UR3
(нужно ≤ 37 rpm, а SHA‑SG дают 59.4 / 55.4 / 47.5 rpm).

**Базовое сопоставление (одна эталонная модель):**

- `shoulder_pan_joint` → SHA32A‑101
- `shoulder_lift_joint` → SHA32A‑101
- `elbow_joint` → SHA25A‑101
- `wrist_1_joint` → SHA20A‑101
- `wrist_2_joint` → SHA20A‑101
- `wrist_3_joint` → SHA20A‑101

Для исследования пространства конфигураций можно разрешить:

- на локоть: {SHA25A‑101, SHA32A‑101}
- на каждое запястье: {SHA20A‑101, SHA25A‑101}

Тогда всего комбинаций: $(1 \times 1 \times 2 \times 2 \times 2 \times 2 = 16$).


## 4. Reflected inertia and armature

В MuJoCo параметр `armature` у сустава — это дополнительная инерция
относительно оси сустава (часто: ротор двигателя + редуктор,
приведённые к координате сустава).

Каталог SHA‑SG даёт `Moment of Inertia GD²/4` в кг·м² относительно
**выходного вала**, который совпадает с осью сустава.
Поэтому в модели мы **не перемножаем на N²**, а используем значения как есть.

Для каждого актуатора сохраняем:

```python
armature_inertia = <GD^2 / 4 из каталога, кг·м^2>
```

и записываем в MJCF:

```python
joint_el.set("armature", f"{hd.armature_inertia:.6e}")
```

В базовом сопоставлении:

- плечи: `armature = 8.0` kg·m²
- локоть: `armature = 2.2` kg·m²
- запястья: `armature = 0.91` kg·m²


## 5.

### 5.1.возьмём мотор как цилиндр

Мотор считаем сплошным цилиндром с осью вдоль локальной оси Z звена.

Для цилиндра массой \(m\), радиуса \(R\) и длины \(h\),
в собственной системе координат (ось вдоль Z):

$[
I_x = I_y = \frac{1}{12} m(3R^2 + h^2), \quad
I_z = \frac{1}{2} m R^2.
$]

Эта формула будет реализована в функции `cylinder_inertia_diag` (см. 9.3).

### 5.2. Проблема: URDF links уже включают какой‑то мотор

URDF UR3 задаёт инерцию звена так, как будто заводской мотор уже внутри.
Если просто «добавить» SHA‑SG, получится по два мотора на каждый сустав (маф)

Поэтому:

1. Вводим гипотетический старый мотор `MotorShell` для каждого сустава.
2. Вычитаем его вклад из массы и инерции звена.
3. Добавляем новый мотор SHA‑SG вместо него.

### 5.3

Параметры подбираются так, чтобы:

- масса мотора меньше массы звена из URDF,
- масса «голого» звена после вычитания была положительной,
- габариты были физически правдоподобны.

Конкретные числа задаются в разделе 9.1 (словарь `OLD_MOTORS`).

### 5.4. формула обновлённой массы

Обозначения:

- $(M_{\text{URDF}})$ — масса из URDF,
- $(m_{\text{old}})$ — масса старого мотора,
- $(m_{\text{new}})$ — масса нового SHA‑SG.

Тогда:

$\[
M_{\text{clean}} = M_{\text{URDF}} - m_{\text{old}},
\quad$
$M_{\text{final}} = M_{\text{clean}} + m_{\text{new}}.
\]
$
### 5.5. Inertia update formula

Обозначения:

- $(\mathbf I_{\text{URDF}})$ — диагональный тензор инерции из URDF
- $(\mathbf I_{\text{old}})$ — тензор инерции цилиндра старого мотора
- $(\mathbf I_{\text{new}})$ — тензор инерции цилиндра нового мотора

При допущении, что центр масс и оси совпадают:

$[
\mathbf I_{\text{clean}} = \mathbf I_{\text{URDF}} - \mathbf I_{\text{old}},
\quad
\mathbf I_{\text{final}} = \mathbf I_{\text{clean}} + \mathbf I_{\text{new}}.
$]

Конкретные численные результаты будут посчитаны кодом в 9.4–9.5.


## 6. Implementation details

### 6.1. `harmonic_drives.py`

В этом модуле:

- определяется dataclass `HarmonicDrive`;
- задаются три модели SHA‑SG с передаточным числом 101:1
  (SHA20A‑101, SHA25A‑101, SHA32A‑101);
- словарь `joint_variants` описывает,
  какие актуаторы допустимы для каждого сустава UR3.

### 6.2. `generate_models.py`

Основные шаги:

1. Парсинг базовой модели `mjmodel.xml` с помощью `lxml`.
2. Для каждого сустава:
   - найти `<joint>` и его родительский `<body>`;
   - обновить `<inertial>` тела (вычесть старый мотор, добавить SHA‑SG);
   - записать `armature` (инерция мотора);
   - создать `<actuator><general>` с ограничением по моменту (`ctrlrange`).
3. Сохранить один или несколько обновлённых MJCF‑файлов.


## 7. How to run

Базовый сценарий:

```bash
python generate_models.py
```

Скрипт:

1. читает `mjmodel.xml`;
2. строит одну эталонную модель с выбранным набором SHA‑SG,
   например `ur3_hd_single_variant.xml`;
3. по желанию генерирует все допустимые комбинации
   в подкаталог `generated_models/`.

Полученные `.xml` можно открыть в MuJoCo Viewer.


## 8. Limitations and assumptions

1. Параметры заводских моторов UR3 неизвестны, поэтому
   `MotorShell` задаются эвристически.
2. Центр масс звена и моторов (старого и нового) считаем совпадающим,
   оси инерции — выровненными (в простом варианте без теоремы Штейнера).
3. Значения `GD²/4` из каталога использованы как есть
   для `armature` (инерция относительно выходного вала).
4. Трение, люфты, гибкость и реальные контроллеры в модель не включены.

Несмотря на упрощения, модель захватывает ключевой эффект:
более тяжёлые звенья, реалистичные лимиты момента
и добавленную инерцию моторов.


## 9. Приложение: полный расчёт в виде кода

В этом разделе — **все вычисления**, которые использовались при выборе актуаторов
и пересчёте масс/инерций. Формат:

> «нам нужно получить … потому что …» + блок кода,  
> который можно запустить и увидеть числа для каждого сустава.

Все блоки можно последовательно выполнить в этом ноутбуке.


### 9.1. URDF‑данные и параметры из каталога

**Что нужно получить**

Собрать в одном месте:

- предельные моменты и скорости суставов из URDF;
- массы и инерции звеньев из URDF;
- параметры SHA‑SG из каталога;
- параметры гипотетических старых моторов (`MotorShell`).

**Почему**

Это исходные данные для всех дальнейших проверок:

- по моменту и скорости (двойное неравенство «требуемое ≤ допустимое»),
- по пересчёту масс и инерций.


In [ ]:
import math
import numpy as np
from dataclasses import dataclass

# --- 1) Joint specs from URDF (effort & velocity) ---

JOINT_URDF = {
    "shoulder": {"tau": 330.0, "omega": 2.16},  # N·m, rad/s
    "elbow":    {"tau": 150.0, "omega": 3.15},
    "wrist":    {"tau":  54.0, "omega": 3.20},
}

# --- 2) Link inertial data from UR3 URDF/MJCF ---

LINK_URDF = {
    "shoulder_link": (
        2.0,  # mass, kg
        np.array([
            0.0080931634294,
            0.0080931634294,
            0.005625,
        ])
    ),
    "upper_arm_link": (
        3.42,
        np.array([
            0.0217284832211,
            0.0217284832211,
            0.00961875,
        ])
    ),
    "forearm_link": (
        1.26,
        np.array([
            0.00654680644378,
            0.00654680644378,
            0.00354375,
        ])
    ),
    "wrist_1_link": (
        0.80,
        np.array([
            0.002084999166,
            0.002084999166,
            0.00225,
        ])
    ),
    "wrist_2_link": (
        0.80,
        np.array([
            0.002084999166,
            0.002084999166,
            0.00225,
        ])
    ),
    "wrist_3_link": (
        0.35,
        np.array([
            0.000912187135125,
            0.000912187135125,
            0.000984375,
        ])
    ),
}

# --- 3) SHA-SG actuators (ratio 101:1) ---

SHA = {
    "SHA20A_101": {
        "tau_max": 107.0,   # N·m
        "n_max":   59.4,    # rpm
        "J_arm":    0.91,   # kg·m^2 (GD^2/4)
        "mass":     2.0,    # kg
        "R":        0.047,  # m (A/2)
        "h":        0.103,  # m (E)
    },
    "SHA25A_101": {
        "tau_max": 204.0,
        "n_max":   55.4,
        "J_arm":    2.2,
        "mass":     5.0,
        "R":        0.057,
        "h":        0.109,
    },
    "SHA32A_101": {
        "tau_max": 433.0,
        "n_max":   47.5,
        "J_arm":    8.0,
        "mass":     2.95,
        "R":        0.073,
        "h":        0.125,
    },
}

# --- 4) Hypothetical old motors inside UR3 links ---

@dataclass
class MotorShell:
    mass: float
    R: float
    h: float

OLD_MOTORS = {
    "shoulder_pan_joint": MotorShell(mass=1.0,  R=0.04,  h=0.08),
    "shoulder_lift_joint":MotorShell(mass=1.0,  R=0.04,  h=0.08),
    "elbow_joint":        MotorShell(mass=0.6,  R=0.035, h=0.07),
    "wrist_1_joint":      MotorShell(mass=0.3,  R=0.025, h=0.05),
    "wrist_2_joint":      MotorShell(mass=0.3,  R=0.025, h=0.05),
    "wrist_3_joint":      MotorShell(mass=0.15, R=0.020, h=0.04),
}

# --- 5) Mapping joint -> link, joint type, chosen actuator ---

JOINT2LINK = {
    "shoulder_pan_joint": "shoulder_link",
    "shoulder_lift_joint": "upper_arm_link",
    "elbow_joint": "forearm_link",
    "wrist_1_joint": "wrist_1_link",
    "wrist_2_joint": "wrist_2_link",
    "wrist_3_joint": "wrist_3_link",
}

JOINT_TYPE = {
    "shoulder_pan_joint": "shoulder",
    "shoulder_lift_joint": "shoulder",
    "elbow_joint": "elbow",
    "wrist_1_joint": "wrist",
    "wrist_2_joint": "wrist",
    "wrist_3_joint": "wrist",
}

# baseline mapping UR3 joint -> SHA-SG model
JOINT2SHA = {
    "shoulder_pan_joint": "SHA32A_101",
    "shoulder_lift_joint": "SHA32A_101",
    "elbow_joint": "SHA25A_101",
    "wrist_1_joint": "SHA20A_101",
    "wrist_2_joint": "SHA20A_101",
    "wrist_3_joint": "SHA20A_101",
}

print("Data structures initialised.")


Data structures initialised.


### 9.2. Проверка подходит ли этот Harmonic Drive по моменту и скорости

**Что нужно получить**

Для каждого типа сустава:

1. требуемый момент $(\tau_{\text{need}}$) с запасом;
2. требуемую максимальную скорость $(n_{\text{need}}$) (rpm) с запасом;
3. проверку двойного неравенства:

$[
\tau_{\text{need}} \le \tau_{\max}^{\text{HD}}, \quad
n_{\text{need}} \le n_{\max}^{\text{HD}}.
$]

 реальная траектория и её худший случай
должны лежать внутри католожных ограничений актуатора (с запасом).


In [ ]:
def radps_to_rpm(omega_rad_s: float) -> float:
    """Convert rad/s to rpm."""
    return omega_rad_s * 60.0 / (2.0 * math.pi)

# safety margins
K_TAU = 1.3
K_OMEGA = 1.2

print("== Torque / speed requirements with safety margins ==")
for jt, spec in JOINT_URDF.items():
    tau_need = K_TAU * spec["tau"]
    n_urdf = radps_to_rpm(spec["omega"])
    n_need = K_OMEGA * n_urdf
    print(f"{jt:8s}: tau_need = {tau_need:7.2f} N·m, "
          f"n_need = {n_need:6.2f} rpm")

print("\n== Check tau_need <= tau_max, n_need <= n_max for each joint ==")
for joint_name, sha_name in JOINT2SHA.items():
    jt = JOINT_TYPE[joint_name]
    tau_need = K_TAU * JOINT_URDF[jt]["tau"]
    n_need = K_OMEGA * radps_to_rpm(JOINT_URDF[jt]["omega"])

    tau_max = SHA[sha_name]["tau_max"]
    n_max   = SHA[sha_name]["n_max"]

    ok_tau = tau_need <= tau_max
    ok_n   = n_need <= n_max

    print(f"{joint_name:18s} -> {sha_name:10s}: "
          f"tau_need={tau_need:6.1f} <= tau_max={tau_max:6.1f} : {ok_tau}, "
          f"n_need={n_need:6.2f} <= n_max={n_max:6.2f} : {ok_n}")


== Torque / speed requirements with safety margins ==
shoulder: tau_need =  429.00 N·m, n_need =  24.75 rpm
elbow   : tau_need =  195.00 N·m, n_need =  36.10 rpm
wrist   : tau_need =   70.20 N·m, n_need =  36.67 rpm

== Check tau_need <= tau_max, n_need <= n_max for each joint ==
shoulder_pan_joint -> SHA32A_101: tau_need= 429.0 <= tau_max= 433.0 : True, n_need= 24.75 <= n_max= 47.50 : True
shoulder_lift_joint -> SHA32A_101: tau_need= 429.0 <= tau_max= 433.0 : True, n_need= 24.75 <= n_max= 47.50 : True
elbow_joint        -> SHA25A_101: tau_need= 195.0 <= tau_max= 204.0 : True, n_need= 36.10 <= n_max= 55.40 : True
wrist_1_joint      -> SHA20A_101: tau_need=  70.2 <= tau_max= 107.0 : True, n_need= 36.67 <= n_max= 59.40 : True
wrist_2_joint      -> SHA20A_101: tau_need=  70.2 <= tau_max= 107.0 : True, n_need= 36.67 <= n_max= 59.40 : True
wrist_3_joint      -> SHA20A_101: tau_need=  70.2 <= tau_max= 107.0 : True, n_need= 36.67 <= n_max= 59.40 : True


### 9.3. Вычисление инерции цилиндра


Функция, которая считает диагональный тензор инерции цилиндра
в собственной системе координат:

$[
I_x = I_y = \frac{1}{12} m(3R^2 + h^2), \quad
I_z = \frac{1}{2} m R^2.
$]

Эта формула дана в задании и используется:

- для вычисления инерции старых моторов (`MotorShell`);
- для вычисления инерции новых SHA‑SG по каталогу.


In [ ]:
def cylinder_inertia_diag(m: float, R: float, h: float) -> np.ndarray:
    """Diagonal inertia of a solid cylinder in its COM frame (axis along Z)."""
    Ixy = (1.0 / 12.0) * m * (3.0 * R**2 + h**2)
    Iz  = 0.5 * m * R**2
    return np.array([Ixy, Ixy, Iz])

# quick check on one motor
old = OLD_MOTORS["shoulder_pan_joint"]
print("Old shoulder motor inertia diag:", cylinder_inertia_diag(old.mass, old.R, old.h))


Old shoulder motor inertia diag: [0.00093333 0.00093333 0.0008    ]


### 9.4. Вычитание старого мотора из инерции звена


Для каждого звена:

1. массу «чистого» звена $(M_{\text{clean}}$);
2. инерцию «чистого» звена $(\mathbf I_{\text{clean}}$).

Формулы (при совпадающих COM и осях (я взял чтобы не усложнять жизнь)):

$[
M_{\text{clean}} = M_{\text{URDF}} - m_{\text{old}}, \quad
\mathbf I_{\text{clean}} = \mathbf I_{\text{URDF}} - \mathbf I_{\text{old}}.
$]


(URDF задаёт звено + старый мотор. Чтобы честно заменить мотор,
сначала удаляем его вклад)


In [ ]:
print("== Remove old motor from link inertia ==")

clean_data = {}  # сохраним для следующего шага

for joint_name, link_name in JOINT2LINK.items():
    M_urdf, I_urdf = LINK_URDF[link_name]
    old_motor = OLD_MOTORS[joint_name]

    I_old = cylinder_inertia_diag(old_motor.mass, old_motor.R, old_motor.h)
    M_clean = M_urdf - old_motor.mass
    I_clean = I_urdf - I_old

    clean_data[joint_name] = (M_clean, I_clean)

    print(f"{joint_name:18s} / {link_name:15s}")
    print(f"  M_urdf  = {M_urdf:8.4f} kg")
    print(f"  m_old   = {old_motor.mass:8.4f} kg")
    print(f"  M_clean = {M_clean:8.4f} kg")
    print(f"  I_urdf  = {I_urdf}")
    print(f"  I_old   = {I_old}")
    print(f"  I_clean = {I_clean}\n")

    assert M_clean > 0.0, "clean mass must stay positive"


== Remove old motor from link inertia ==
shoulder_pan_joint / shoulder_link  
  M_urdf  =   2.0000 kg
  m_old   =   1.0000 kg
  M_clean =   1.0000 kg
  I_urdf  = [0.00809316 0.00809316 0.005625  ]
  I_old   = [0.00093333 0.00093333 0.0008    ]
  I_clean = [0.00715983 0.00715983 0.004825  ]

shoulder_lift_joint / upper_arm_link 
  M_urdf  =   3.4200 kg
  m_old   =   1.0000 kg
  M_clean =   2.4200 kg
  I_urdf  = [0.02172848 0.02172848 0.00961875]
  I_old   = [0.00093333 0.00093333 0.0008    ]
  I_clean = [0.02079515 0.02079515 0.00881875]

elbow_joint        / forearm_link   
  M_urdf  =   1.2600 kg
  m_old   =   0.6000 kg
  M_clean =   0.6600 kg
  I_urdf  = [0.00654681 0.00654681 0.00354375]
  I_old   = [0.00042875 0.00042875 0.0003675 ]
  I_clean = [0.00611806 0.00611806 0.00317625]

wrist_1_joint      / wrist_1_link   
  M_urdf  =   0.8000 kg
  m_old   =   0.3000 kg
  M_clean =   0.5000 kg
  I_urdf  = [0.002085 0.002085 0.00225 ]
  I_old   = [1.09375e-04 1.09375e-04 9.37500e-05]
  I_c

### 9.5. Добавление нового SHA‑SG к звену


Итоговые масса и инерция звена с новым мотором:

$[
M_{\text{final}} = M_{\text{clean}} + m_{\text{new}}, \quad
\mathbf I_{\text{final}} = \mathbf I_{\text{clean}} + \mathbf I_{\text{new}},
$]

где $(\mathbf I_{\text{new}}$) — инерция цилиндра SHA‑SG.

Эти параметры затем записываются в `<inertial>` в MJCF.


In [ ]:
print("== Add new SHA-SG motor and get final masses & inertias ==")

final_data = {}

for joint_name, link_name in JOINT2LINK.items():
    M_clean, I_clean = clean_data[joint_name]
    sha_name = JOINT2SHA[joint_name]
    sha = SHA[sha_name]

    I_new = cylinder_inertia_diag(sha["mass"], sha["R"], sha["h"])
    M_final = M_clean + sha["mass"]
    I_final = I_clean + I_new

    final_data[joint_name] = (M_final, I_final)

    print(f"{joint_name:18s} -> {sha_name:10s} / {link_name:15s}")
    print(f"  M_clean = {M_clean:10.4f} kg")
    print(f"  m_new   = {sha['mass']:10.4f} kg")
    print(f"  M_final = {M_final:10.4f} kg")
    print(f"  I_clean = {I_clean}")
    print(f"  I_new   = {I_new}")
    print(f"  I_final = {I_final}\n")


== Add new SHA-SG motor and get final masses & inertias ==
shoulder_pan_joint -> SHA32A_101 / shoulder_link  
  M_clean =     1.0000 kg
  m_new   =     2.9500 kg
  M_final =     3.9500 kg
  I_clean = [0.00715983 0.00715983 0.004825  ]
  I_new   = [0.00777128 0.00777128 0.00786027]
  I_final = [0.01493111 0.01493111 0.01268527]

shoulder_lift_joint -> SHA32A_101 / upper_arm_link 
  M_clean =     2.4200 kg
  m_new   =     2.9500 kg
  M_final =     5.3700 kg
  I_clean = [0.02079515 0.02079515 0.00881875]
  I_new   = [0.00777128 0.00777128 0.00786027]
  I_final = [0.02856643 0.02856643 0.01667903]

elbow_joint        -> SHA25A_101 / forearm_link   
  M_clean =     0.6600 kg
  m_new   =     5.0000 kg
  M_final =     5.6600 kg
  I_clean = [0.00611806 0.00611806 0.00317625]
  I_new   = [0.00901167 0.00901167 0.0081225 ]
  I_final = [0.01512972 0.01512972 0.01129875]

wrist_1_joint      -> SHA20A_101 / wrist_1_link   
  M_clean =     0.5000 kg
  m_new   =     2.0000 kg
  M_final =     2.5000 k

### 9.6. Проверка armature для каждого сустава

нужно убедиться что для каждого сустава в `armature` мы будем писать
ровно значение `GD²/4` из каталога.


In [ ]:
print("== Armature values per joint ==")

for joint_name, sha_name in JOINT2SHA.items():
    J_arm = SHA[sha_name]["J_arm"]
    print(f"{joint_name:18s} -> {sha_name:10s}: armature = {J_arm:.3f} kg·m^2")


== Armature values per joint ==
shoulder_pan_joint -> SHA32A_101: armature = 8.000 kg·m^2
shoulder_lift_joint -> SHA32A_101: armature = 8.000 kg·m^2
elbow_joint        -> SHA25A_101: armature = 2.200 kg·m^2
wrist_1_joint      -> SHA20A_101: armature = 0.910 kg·m^2
wrist_2_joint      -> SHA20A_101: armature = 0.910 kg·m^2
wrist_3_joint      -> SHA20A_101: armature = 0.910 kg·m^2


### 9.7.
Всё, что посчитано в разделе 9:

- используется в скрипте `generate_models.py` при:
  - вычитании старых моторов из звеньев,
  - добавлении SHA‑SG,
  - заполнении `armature` и `ctrlrange`;
- проходит проверки:
  - двойное неравенство по моменту и скорости,
  - положительность масс и инерций.
